In [1]:
import galprime
import sys
import os
from astropy.io import fits
from astropy.table import Table
from numpy.random import normal
import numpy as np

from scipy.interpolate import interp1d

from tqdm import tqdm
from matplotlib import pyplot as plt

import importlib
importlib.reload(galprime)

import dask.bag as db

In [2]:

config = galprime.load_config_file('../newconfig.galprime')
config["CORES"] = 2
# galprime.print_config(config)
# sys.exit()

# Load in backgrounds and psfs
bgs = galprime.cutouts_from_file("../cutouts.fits")
psfs = galprime.cutouts_from_file('../i_psfs.fits')

# TODO change this to the actual COSMOS mag table once I have it available to me

cosmos_table = Table.read("../COSMOS_V9_MAGS.fits", format="fits")

mags = cosmos_table["i"][cosmos_table["i"] > 0]

mag_kde = galprime.gaussian_kde(mags)

gp = galprime.GPrime(config='../newconfig.galprime', backgrounds=bgs, mag_kde=mag_kde, psfs=psfs)


# test_container.save_outputs(filename="GPrime_" + str(test_container.metadata["ID"]) + ".fits")

container_list = gp.pipeline(max_bins=1, debug=True, process_method=galprime.gprime_single)



Running GalPRIME, version 2.1.2
-12 bins in total. Running GalPRIME on 1 bins total
-Backgrounds were supplied: 50 in total.
Working through 20 models for bin: [ 9.6 10.   0.1  0.3  0.   0.5] with 2 cores.
['out_sim_test/tempfiles/container_18.gpobj', 'out_sim_test/tempfiles/container_8.gpobj', 'out_sim_test/tempfiles/container_19.gpobj', 'out_sim_test/tempfiles/container_9.gpobj', 'out_sim_test/tempfiles/container_11.gpobj', 'out_sim_test/tempfiles/container_1.gpobj', 'out_sim_test/tempfiles/container_3.gpobj', 'out_sim_test/tempfiles/container_13.gpobj', 'out_sim_test/tempfiles/container_17.gpobj', 'out_sim_test/tempfiles/container_7.gpobj', 'out_sim_test/tempfiles/container_5.gpobj', 'out_sim_test/tempfiles/container_15.gpobj', 'out_sim_test/tempfiles/container_2.gpobj', 'out_sim_test/tempfiles/container_12.gpobj', 'out_sim_test/tempfiles/container_10.gpobj', 'out_sim_test/tempfiles/container_0.gpobj', 'out_sim_test/tempfiles/container_4.gpobj', 'out_sim_test/tempfiles/container_14.

In [37]:
import dask
from dask.distributed import Client

In [38]:
client = Client(lifetime=120,  n_workers=2)

client
dask.config.config

{'temporary-directory': None,
 'tokenize': {'ensure-deterministic': False},
 'dataframe': {'shuffle-compression': None,
  'parquet': {'metadata-task-size-local': 512,
   'metadata-task-size-remote': 16}},
 'array': {'svg': {'size': 120}, 'slicing': {'split-large-chunks': None}},
 'optimization': {'fuse': {'active': None,
   'ave-width': 1,
   'max-width': None,
   'max-height': inf,
   'max-depth-new-edges': None,
   'subgraphs': None,
   'rename-keys': True}},
 'distributed': {'version': 2,
  'scheduler': {'allowed-failures': 3,
   'bandwidth': 100000000,
   'blocked-handlers': [],
   'default-data-size': '1kiB',
   'events-cleanup-delay': '1h',
   'idle-timeout': None,
   'transition-log-length': 100000,
   'events-log-length': 100000,
   'work-stealing': True,
   'work-stealing-interval': '100ms',
   'worker-ttl': None,
   'pickle': True,
   'preload': [],
   'preload-argv': [],
   'unknown-task-duration': '500ms',
   'default-task-durations': {'rechunk-split': '1us', 'split-shuffle

In [35]:
def load_and_process(filename):
    container = galprime.load_pickle(filename)
    try:
        container.process_object(plot=False)
        print("Completed:", filename)
    except Exception as error:
        print("Failed:", filename, error)
    return container

A = client.map(load_and_process, container_list)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [36]:
client.gather(A)

out_sim_test/tempfiles/container_9.gpobj segment_img must be a SegmentationImage
out_sim_test/tempfiles/container_18.gpobj segment_img must be a SegmentationImage
out_sim_test/tempfiles/container_1.gpobj All boxes contain > 1587.6000000000001 (90 percent per box) masked pixels (or all are completely masked). Please check your data or increase "exclude_percentile" to allow more boxes to be included.


tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x1122a13d0>>, <Task finished name='Task-19087' coro=<Worker.close_gracefully() done, defined at /Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/worker.py:1730> exception=OSError('Timed out during handshake while connecting to tcp://127.0.0.1:55167 after 30 s')>)
Traceback (most recent call last):
  File "/Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/comm/tcp.py", line 211, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hsouch/opt/miniconda3/lib/python3.9/asyncio/tasks.py", line 490, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exce

Completed: out_sim_test/tempfiles/container_10.gpobj


tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x1122013d0>>, <Task finished name='Task-19030' coro=<Worker.close_gracefully() done, defined at /Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/worker.py:1730> exception=OSError('Timed out during handshake while connecting to tcp://127.0.0.1:56458 after 30 s')>)
Traceback (most recent call last):
  File "/Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/comm/tcp.py", line 211, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hsouch/opt/miniconda3/lib/python3.9/asyncio/tasks.py", line 490, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exce

Completed: out_sim_test/tempfiles/container_0.gpobj
out_sim_test/tempfiles/container_0.gpobj All boxes contain > 1587.6000000000001 (90 percent per box) masked pixels (or all are completely masked). Please check your data or increase "exclude_percentile" to allow more boxes to be included.


KeyboardInterrupt: 